In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("reviews_kkstream.csv")
# create the label
df["is_bad_review"] = df["Reviewer Ratings"].apply(lambda x: 0 if int(x) <= 3 else 1)
# select only relevant columns
df = df[["Review Body", "is_bad_review"]]
df.head()

,Review Body,is_bad_review
0,歌詞見れるいいずれ誤字あるあとお気に入りプレイリスト開くライブラリ更新リセットれるマジ入れラ...,0
1,通知切る方法分かりすぎる見る新たアプリ入れ通知切れ判明アプリ開発若者毎日アクティブ新曲買っ聴...,0
2,どうしてもLISMO比べダウンロード反映LISMO動画一覧表示パス分離とにかく使いLISMO...,0
3,以前購入機種だぶっダウンロードれる消す出来機種するダウンロード出来有るガラケー購入スマ出来有り,0
4,LISMOライブラリ開けなっ愛着あっLISMO使っ消し下らないうたパスLISMOいらついて最...,0


In [4]:
sentences = df['Review Body'].apply(str).values
y = df['is_bad_review'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.20, random_state=1000)

In [5]:
import tensorflow as tf
import numpy
import keras
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
from keras_adabound import AdaBound
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.layers import Convolution1D, MaxPooling1D
from keras.engine import Input
from keras.layers import CuDNNLSTM, Embedding, SpatialDropout1D, Bidirectional
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.preprocessing import text,sequence
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
import pandas
import os
from gensim.models.word2vec import Word2Vec
from collections import Counter

Using TensorFlow backend.


In [6]:
# Input parameters
max_features = 5000
max_len = 200
embedding_size = 300

# Convolution parameters
filter_length = 3
nb_filter = 150
pool_length = 2
cnn_activation = 'relu'
border_mode = 'same'

# RNN parameters
output_size = 50
rnn_activation = 'tanh'
recurrent_activation = 'hard_sigmoid'

# Compile parameters
loss = 'binary_crossentropy'
optimizer = 'rmsprop'

# Training parameters
batch_size = 128
nb_epoch = 250
validation_split = 0.25
shuffle = True

In [7]:
import gensim
w2v = gensim.models.Word2Vec.load('ja.bin')

In [8]:
# Build vocabulary & sequences
tk = text.Tokenizer(num_words = max_features, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = " ")
tk.fit_on_texts(sentences)
x = tk.texts_to_sequences(sentences)
word_index = tk.word_index
x = sequence.pad_sequences(x,maxlen=max_len)

In [9]:
word_vectors = w2v.wv
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200
WV_DIM = 300
nb_words = min(MAX_NB_WORDS, len(word_vectors.vocab))
vocab = Counter()
word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}

# we initialize the matrix with random numbers
import numpy as np
wv_matrix = (np.random.rand(nb_words, WV_DIM) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass
    
embedding_layer = Embedding(nb_words, 
                     WV_DIM, 
                     mask_zero = False, 
                     weights = [wv_matrix], 
                     input_length = MAX_SEQUENCE_LENGTH, 
                     trainable = False)

In [10]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(Convolution1D(
                        filters=nb_filter,
                        kernel_size=filter_length,
                        border_mode=border_mode,
                        activation=cnn_activation,
                        subsample_length=1))
model.add(MaxPooling1D(pool_size=pool_length))
model.add(LSTM(units=output_size,activation=rnn_activation,recurrent_activation=recurrent_activation))
model.add(Dense(5))
model.add(Dropout(0.25))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = loss, optimizer = AdaBound(lr = 1e-3, final_lr = 0.1), metrics = ['accuracy'])

print('CNN + LSTM')

path = 'weights.hdf5'
model_checkpoint = ModelCheckpoint(path, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'auto')
early_stopping = EarlyStopping(monitor = 'loss', patience = 8, verbose = 1, mode = 'auto')

history = model.fit(
              x, y, batch_size = batch_size, 
              epochs = nb_epoch, 
              validation_split = validation_split, 
              shuffle = shuffle, 
              verbose = 1, 
              callbacks = [model_checkpoint, early_stopping])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=150, kernel_size=3, activation="relu", strides=1, padding="same")`
  if __name__ == '__main__':


IndexError: tuple index out of range